In [1]:
from glob import glob
import scipy
import numpy as np
import matplotlib.pyplot as plt
from dataloader import MultiStreamLoader, concat_variable_length_files, get_data_loader

import torch
from torch import nn
from torch import optim

from model import UNet

from tensorboardX import SummaryWriter

In [2]:
writer = SummaryWriter()

In [3]:
wav_files = sorted(glob('./data/background/YD/*.wav'))
background_stream_loader = MultiStreamLoader(wav_files)

speech_files = sorted(glob('./data/speech/KSS/*/*.wav'))
before_folding = len(speech_files)
speech_files = concat_variable_length_files(speech_files)
print(f'{before_folding:5} => {len(speech_files):5}')

background_stream_loader = MultiStreamLoader(wav_files)
dataloader = get_data_loader([f[0] for f in speech_files], background_stream_loader)

100%|██████████| 12854/12854 [00:01<00:00, 8369.89it/s]

12854 =>  6531


In [4]:
'''
Our implementation of U-Net is similar to that of [11]. 
Each encoder layer consists of a strided 2D convolution of stride 2 and kernel size 5x5, 
batch normalization, and leaky rectified linear units (ReLU) with leakiness 0.2. 
In the decoder we use strided deconvolution (sometimes referred to as transposed convolution) 
with stride 2 and kernel size 5x5, batch normalization, plain ReLU, 
and use 50% dropout to the first three layers, as in [11]. 
In the final layer we use a sigmoid activation function. 
The model is trained using the ADAM [12] optimizer.
'''


l1_loss = nn.L1Loss(reduction='mean')
l2_loss = nn.MSELoss(reduction='mean')

model = UNet()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

for i in range(100):

    for i, batch in enumerate(dataloader):
        Zxxs, log_spectrograms = batch

        # Zxxs[0].shape
        # torch.Size([4, 321, 1025, 2])
        # log_spectrograms[0].shape
        # torch.Size([4, 321, 1025])
        signal, noise, mixed = log_spectrograms

        output = model(mixed)
        loss1 = l1_loss(output, signal)
        loss2 = l2_loss(output, signal)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(loss)

tensor(0.1488, grad_fn=<AddBackward0>)
tensor(0.1497, grad_fn=<AddBackward0>)
tensor(0.1509, grad_fn=<AddBackward0>)
tensor(0.1522, grad_fn=<AddBackward0>)
tensor(0.1473, grad_fn=<AddBackward0>)
tensor(0.1405, grad_fn=<AddBackward0>)
tensor(0.1456, grad_fn=<AddBackward0>)
tensor(0.1201, grad_fn=<AddBackward0>)
tensor(0.1430, grad_fn=<AddBackward0>)
tensor(0.1452, grad_fn=<AddBackward0>)
tensor(0.1124, grad_fn=<AddBackward0>)
tensor(0.1391, grad_fn=<AddBackward0>)
tensor(0.1427, grad_fn=<AddBackward0>)
tensor(0.1075, grad_fn=<AddBackward0>)
tensor(0.1113, grad_fn=<AddBackward0>)
tensor(0.1373, grad_fn=<AddBackward0>)
tensor(0.1106, grad_fn=<AddBackward0>)
tensor(0.1271, grad_fn=<AddBackward0>)
tensor(0.1340, grad_fn=<AddBackward0>)
tensor(0.1250, grad_fn=<AddBackward0>)
tensor(0.1098, grad_fn=<AddBackward0>)
tensor(0.1355, grad_fn=<AddBackward0>)
tensor(0.1257, grad_fn=<AddBackward0>)
tensor(0.1258, grad_fn=<AddBackward0>)
tensor(0.1080, grad_fn=<AddBackward0>)
tensor(0.1302, grad_fn=<A

Traceback (most recent call last):
  File "/Users/joowhan/opt/anaconda3/envs/py3_env/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/joowhan/opt/anaconda3/envs/py3_env/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/joowhan/opt/anaconda3/envs/py3_env/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/joowhan/opt/anaconda3/envs/py3_env/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 